In [ ]:
using Revise
using BenchmarkTools

using Kwat

In [ ]:
tr = joinpath("..", "tree", "")

;

In [ ]:
em = "1.empty.md"

he_ = [li for li in readlines(em) if li != ""]

;

In [ ]:
using Kwat.path: clean

function is_good_structure(na::String)::Bool
    
    return occursin(r"^([0-9]+\.){1,2}[_0-9a-z]+(\.md$|$)", na)
    
end

In [ ]:
function split_good_structure(na::String)::Vector{Union{Float64,String}}

    pa_ = Vector{Union{Float64,String}}()

    sp_ = string.(split(na, "."))

    it1 = tryparse(Int64, sp_[1])

    it2 = tryparse(Int64, sp_[2])

    if isnothing(it2)

        pa_ = [convert(Float64, it1); sp_[2:end]]

    else

        pa_ = [parse(Float64, join([it1, it2], ".")); sp_[3:end]]

    end

    return pa_

end

In [ ]:
for (ro, di_, fi_) in walkdir(tr; topdown = false)

    fi_ = [fi for fi in fi_ if !startswith(fi, ".")]

    for fi in fi_

        cl = clean(fi)

        if fi != cl

            println(fi, " ==> ", cl)

            mv(joinpath(ro, fi), joinpath(ro, cl))

        end

    end

    for fi in fi_

        pa = joinpath(ro, fi)

        li_ = readlines(pa)

        if length(li_) == 0

            println("Is empty (populating): ", pa)

            cp(em, pa; force = true)

        elseif !all(he in li_ for he in he_)

            error("Is missing at least 1 required header: ", pa)

        elseif !all(0 .< diff([findfirst(li_ .== he) for he in he_]))

            error("Has incorrectly ordered headers: ", pa)

        end

    end

    go_ = Vector{String}()

    ba_ = Vector{String}()

    for na in [di_; fi_]

        if is_good_structure(na)

            push!(go_, na)

        else

            push!(ba_, na)

        end

    end

    sort!(go_; by = split_good_structure)

    n_go = length(go_)

    for (id, na) in enumerate([go_; ba_])

        pa1 = joinpath(ro, na)

        if id <= n_go

            pa2 = joinpath(ro, join([id; split_good_structure(na)[2:end]], "."))

        else

            pa2 = joinpath(ro, join([id, na], "."))

        end

        if pa1 != pa2

            println(splitdir(pa1)[2], " ==> ", splitdir(pa2)[2])

            mv(pa1, pa2)

        end

    end

end